In [1]:
import langchain
from langchain_cohere import ChatCohere
from langchain_community.document_loaders import PyPDFLoader , DirectoryLoader 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import LLMChain
from langchain.retrievers import PineconeHybridSearchRetriever
from langchain.memory import ConversationBufferWindowMemory
from pinecone_text.sparse import BM25Encoder
from langchain.vectorstores import FAISS

c:\Users\admin\anaconda3\envs\medicalchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
llm = ChatCohere(cohere_api_key="" , temperature=0.7)

In [3]:
llm

ChatCohere(client=<cohere.client.Client object at 0x000001A3A2E603D0>, async_client=<cohere.client.AsyncClient object at 0x000001A3A2E60DF0>, temperature=0.7, cohere_api_key=SecretStr('**********'))

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data , glob="*.pdf",loader_cls=PyPDFLoader)
    document = loader.load()

    return document

In [7]:
data = load_pdf("C:\Sarthi\Chatbot\data")

In [8]:
data 

[Document(metadata={'source': 'C:\\Sarthi\\Chatbot\\data\\Ahemdabad.pdf', 'page': 0}, page_content='Ahmedabad hosts  several  reputable  engineering  colleges.  Here  are \nsome  of the top engineering institutions:  \n \n1. ADANI UNIVERSITY (AU -FEST)  \n2. NIRMA UNIVERSITY OF SCIENCE & TECHNOLOGY  \n3. AHMEDABAD UNIVERSITY (AU)   \n4. L.D.COLLEGE OF ENGINEERING  \n5. L.J. INSTITUTE OF ENGG AND TECH  \n6. ADITYA SILVER OAK INSTITUTE  \n7. AHMEDABAD INSTITUTE OF TECH  \n8. APOLLO INSTITUTE OF ENGINEERING & TECHNOLOGY  \n9. FACULTY OF ENGINEERING & TECHNOLOGY - GLS UNIVERSITY  \n10. FACULTY OF TECHNOLOGY, CEPT UNIVERSITY  \n11. GUJARAT TECHNOLOGICAL UNIVERSITY SCHOOL OF ENGINEERING AND  \nTECHNOLOGY, CHANDKHEDA, AHMEDABAD  \n12. INDUS INSTITUTE OF TECH. & ENGG., RACHARDA, AHMEDABAD  \n13. INSTITUTE OF INFRASTRUCTURE,TECHNOLOGY,RESEARCH AND \nMANAGEMENT(IITRAM)  \n14. JG UNIVERSITY  \n15. RAI SCHOOL OF ENGINEERING, DHOLKA, AHMEDABAD  \n16. SAL INSTITUTE OF TECH. & ENGG. RESEARCH  \n17. S

In [9]:
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=384 , chunk_overlap=20)
    text_chunk = text_splitter.split_documents(data)

    return text_chunk

In [10]:
text_chunk = text_split(data)

In [11]:
text_chunk

[Document(metadata={'source': 'C:\\Sarthi\\Chatbot\\data\\Ahemdabad.pdf', 'page': 0}, page_content='Ahmedabad hosts  several  reputable  engineering  colleges.  Here  are \nsome  of the top engineering institutions:  \n \n1. ADANI UNIVERSITY (AU -FEST)  \n2. NIRMA UNIVERSITY OF SCIENCE & TECHNOLOGY  \n3. AHMEDABAD UNIVERSITY (AU)   \n4. L.D.COLLEGE OF ENGINEERING  \n5. L.J. INSTITUTE OF ENGG AND TECH  \n6. ADITYA SILVER OAK INSTITUTE  \n7. AHMEDABAD INSTITUTE OF TECH'),
 Document(metadata={'source': 'C:\\Sarthi\\Chatbot\\data\\Ahemdabad.pdf', 'page': 0}, page_content='8. APOLLO INSTITUTE OF ENGINEERING & TECHNOLOGY  \n9. FACULTY OF ENGINEERING & TECHNOLOGY - GLS UNIVERSITY  \n10. FACULTY OF TECHNOLOGY, CEPT UNIVERSITY  \n11. GUJARAT TECHNOLOGICAL UNIVERSITY SCHOOL OF ENGINEERING AND  \nTECHNOLOGY, CHANDKHEDA, AHMEDABAD  \n12. INDUS INSTITUTE OF TECH. & ENGG., RACHARDA, AHMEDABAD  \n13. INSTITUTE OF INFRASTRUCTURE,TECHNOLOGY,RESEARCH AND'),
 Document(metadata={'source': 'C:\\Sarthi\\Cha

In [12]:
len(text_chunk)

1517

In [13]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\admin\AppData\Local\Temp\ipykernel_11732\323160131.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\admin\anaconda3\envs\medicalchatbot\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB

In [ ]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(
        api_key=""
    )
index_name1 = "sarthi"
if index_name1 not in pc.list_indexes().names():
    pc.create_index(
            name=index_name1,
            dimension=384,
            metric='dotproduct',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )        
        )

In [15]:
index = pc.Index(index_name1)

In [16]:
index

In [17]:
bm25encoder = BM25Encoder().default()

In [18]:
bm25encoder.fit([t.page_content for t in text_chunk])

100%|██████████| 1517/1517 [00:01<00:00, 927.18it/s] 


In [19]:
bm25encoder.dump("bm25encoder_values.json")

In [20]:
bm25_encoder = BM25Encoder().load("bm25encoder_values.json")

In [21]:
retreiver = PineconeHybridSearchRetriever(embeddings=embedding , sparse_encoder=bm25_encoder , index=index)

In [22]:
retreiver.add_texts([t.page_content for t in text_chunk])

100%|██████████| 48/48 [01:03<00:00,  1.32s/it]


In [23]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
and do not say as accorrding to context.

CONTEXT: {context}

QUESTION: {question}"""

In [24]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context','question']
)

In [25]:
memory = ConversationBufferWindowMemory(k=10)

In [26]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": prompt}

In [27]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retreiver,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    )

In [26]:
# output = chain.invoke("""what is opening and closing rank  of Computer Engineering for  category in Birla Vishvakarma Maha Vidyalaya (GIA)""")

In [27]:
# print(output['result'])

In [ ]:
while True:
    user_input = input(f"Input Prompt")
    output = chain({'query' : user_input})
    print("Response : " , output['result'])

Response :  The opening rank for Computer Science & Engineering General Category in Faculty of Technology & Engineering, MSU, Vadodara is 73.
Response :  I don't know.
Response :  The opening rank for CSE for the EWS category in MSU is 36150.
Response :  What is the opening rank for Chemical Engineering (TFWS) in the Faculty of Technology & Engineering, MSU, Vadodara?
Response :  I don't know.
Response :  I don't know.
Response :  The opening rank for Computer Science & Engineering General Category in Faculty of Technology & Engineering, MSU, Vadodara is 73.
Response :  PARUL INSTITUTE OF ENGINEERING & TECHNOLOGY has the following branches:

1. Computer Science & Engineering
2. Artificial Intelligence and Data
3. Aeronautical Engineering
4. Electronics & Communication
5. Mathematics and Computing
6. Robotics & Artificial Intelligence
7. Mechanical Engineering
8. Electrical Engineering
Response :  I don't know.
Response :  What is the opening rank for Computer Science & Engineering in t